## Libraries Used

In [ ]:
# Data Handling
import pandas as pd
import numpy as np


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.linear_model import LogisticRegression

# Model Evaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score

# Display Formatting
from IPython.display import display, HTML

## Dataset extraction and organization
### Load and combine all datasets


In [ ]:
import os
import pandas as pd

# Ensure data folder exists
os.makedirs("data", exist_ok=True)

# Load datasets
prices_df = pd.read_csv("data/prices-split-adjusted.csv")
securities_df = pd.read_csv("data/securities.csv")

# Display first few rows (optional in scripts, great for notebooks)
prices_df.head(), securities_df.head()

# Filter IT sector companies
it_companies = securities_df[securities_df['GICS Sector'] == 'Information Technology']

# Select 10 companies (including AAPL)
selected_companies = ['AAPL', 'MSFT', 'ORCL', 'IBM', 'INTC', 'CSCO', 'HPQ', 'ADBE', 'NVDA', 'TXN']
print(f"Selected Companies: {', '.join(selected_companies)}")

# Filter prices
filtered_prices = prices_df[prices_df['symbol'].isin(selected_companies)].copy()
filtered_prices['date'] = pd.to_datetime(filtered_prices['date'])
filtered_prices = filtered_prices.sort_values(by=['symbol', 'date'])
filtered_prices = filtered_prices.dropna().drop_duplicates()
filtered_prices = filtered_prices[['date', 'symbol', 'open', 'close', 'volume']]


# Model 1: Only AAPL  model1_data = df[df['symbol'] == 'AAPL']

# model_1_data = filtered_prices[filtered_prices['symbol'] == 'AAPL']
# model_1_data.to_csv("data/model1_data_AAPL.csv", index=False)

# Model 2: All 10 companies
model_2_data = filtered_prices
model_2_data.to_csv("data/model2_data_all_companies.csv", index=False)

# Confirmation
print("Saved model datasets to 'data/' folder")


## Data Cleaning and Preparation




✅ make the models more flexible. Through some minor changes in preprocessing we can have a model that randomly choose ten companies from a same business area and return the one company model 1 specification and then compare with model 2 as well as we did actually. I think, Jack mentioned a similar idea few days ago. After writing report, if I have extra time I can do it and there is no burden, I think.

In [ ]:
# Code here
df = pd.read_csv('data/model2_data_all_companies.csv')
df.head()

## Exploratory Data Analysis
### Visualizing and analyzing sentiment distribution.

In [ ]:
# Code here

## Feature Engineering

In [ ]:
# Loading the filtered and cleaned data
df = pd.read_csv('data/model2_data_all_companies.csv')

# Converting the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

#Sorting data by company and date( we did it in preprocessing, bbut just to be sure since it is a crucial step)
df= df.sort_values(by=['symbol', 'date'])

# Creating a daily return feature
df['daily_change'] = (df['close'] - df['open']) / df['open'] # measuring the percentage change in the stock price daily for each index

# creating the volum change feature relative to previous day
df['volume_change'] = df.groupby('symbol')['volume'].diff() # Measuring how the volume changed from the previous day.

# Creating a rolling average of the closing price over the past 10 days
df['rolling_close_mean'] = df.groupby('symbol')['close'].transform(lambda x: x.rolling(window=10).mean()) # Each row gets a new value, the 10-day average at that row.

# creaing a rolling average of the volume over the past 10 days
df['rolling_volume_mean'] = df.groupby('symbol')['volume'].transform(lambda x: x.rolling(window=10).mean()) # Each row gets a new value, the 10-day average at that row.

# creating the target variable binary to see the next day's price is higher than today's price or not
df['next_close'] = df.groupby('symbol')['close'].shift(-1) 
df['target'] = (df['next_close'] > df['close']).astype(int) # if row['next_close'] > row['close']: return 1 else: return 0

# Dropping rows with missing values caused by rolling or shifting operations
df = df.dropna() # first 9 rows becuase of roling windows and the last because of shifting operations

# Checking the final result
display(df[['date', 'symbol', 'open', 'close', 'daily_change', 'volume_change', 'rolling_close_mean', 'rolling_volume_mean', 'target']].head())





## Exploratory Data Analysis After feature engineering
### Visualizing and analyzing sentiment distribution.

### Distribution Comparison of Closing Prices and Trading Volumes (AAPL vs. IT Sector)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter Model 1 (AAPL only) and Model 2 (all IT companies)
model_1_data = df[df['symbol'] == 'AAPL']
model_2_data = df  # already contains all 10 companies

# ---- Closing Price Distribution ----
plt.figure(figsize=(12, 5))
sns.histplot(model_1_data['close'], bins=50, label='AAPL', kde=True, color='blue')
sns.histplot(model_2_data['close'], bins=50, label='All IT Companies', kde=True, color='red', alpha=0.5)
plt.xlabel("Closing Price")
plt.ylabel("Frequency")
plt.title("Distribution of Closing Prices (AAPL vs IT Sector)")
plt.legend()
plt.show()

# ---- Trading Volume Distribution ----
plt.figure(figsize=(12, 5))
sns.histplot(model_1_data['volume'], bins=50, label='AAPL', kde=True, color='blue')
sns.histplot(model_2_data['volume'], bins=50, label='All IT Companies', kde=True, color='red', alpha=0.5)
plt.xlabel("Trading Volume")
plt.ylabel("Frequency")
plt.title("Distribution of Trading Volume (AAPL vs IT Sector)")
plt.legend()
plt.show()


### Closing Price Trends Over Time (AAPL vs IT Sector)


In [ ]:
plt.figure(figsize=(12, 6))

# AAPL only
sns.lineplot(data=model_1_data, x='date', y='close', label='AAPL', color='blue')

# All other IT companies (excluding AAPL to avoid duplicate line)
other_companies = model_2_data[model_2_data['symbol'] != 'AAPL']
sns.lineplot(data=other_companies, x='date', y='close', hue='symbol', alpha=0.6)

plt.xlabel("Date")
plt.ylabel("Closing Price")
plt.title("Stock Price Trends Over Time (AAPL vs IT Sector)")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


### Trading Volume Trends Over Time (AAPL vs IT Sector)


In [ ]:
plt.figure(figsize=(12, 6))

# AAPL volume
sns.lineplot(data=model_1_data, x='date', y='volume', label='AAPL', color='blue')

# Volume for other IT companies
sns.lineplot(data=other_companies, x='date', y='volume', hue='symbol', alpha=0.6)

plt.xlabel("Date")
plt.ylabel("Trading Volume")
plt.title("Trading Volume Trends Over Time (AAPL vs IT Sector)")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


### Heatmap of Pairwise Correlations Between Stock Closing Prices (IT Sector)

In [ ]:
# Pivot the data: each column is one company, rows are dates, values are closing prices
pivot_data = df.pivot(index='date', columns='symbol', values='close')

# Compute correlation matrix between the companies
correlation_matrix = pivot_data.corr()

# Plot the heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Correlation Matrix of Stock Closing Prices (IT Sector)")
plt.tight_layout()
plt.show()

### Target Variable Distribution: Price Increase vs. No Increase


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=df, x='target')
plt.title('Target Distribution (0 = price down or same, 1 = price up)')
plt.show()


### Correlation Matrix of Engineered Features and Target Variable


In [ ]:
corr = df[['daily_change', 'volume_change', 'rolling_close_mean', 'rolling_volume_mean', 'target']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Feature Correlation Matrix")
plt.show()


### AAPL Closing Price vs. 10-Day Rolling Average

## Train-Test Split

In [ ]:
# from sklearn.model_selection import train_test_split
# We don't use above library Because:
    # this is time series data (stock prices), maintaining chronological order is critical to avoid data leakage.
    # Instead, we manually split the data using iloc to preserve the time-based structure.


# Filteting Model 1 ( AAPL only ) and Model 2 ( AAPL + others )
# using copy method to avoid unintended changes
model_1_data = df[df['symbol'] == 'AAPL'].copy()
model_2_data = df.copy()

#Dropping unnecessary columns 
drop_cols = ['symbol', 'next_close', 'date', 'open', 'close', 'volume'] # these colums are not featyres for our models
model_1_data = model_1_data.drop(columns=drop_cols)
model_2_data = model_2_data.drop(columns=drop_cols)

# Chronologically splitting data into train and test sets( 80% train, 20% test )
# We don't use sklearn's train_test_split because we want to keep the chronological order because it shuffles the data

#Model 1 (APPL Only)
model_1_split_index = int(len(model_1_data) * 0.8) # 80% index for train set
model_1_train = model_1_data.iloc[:model_1_split_index] # 80% of rows for train set
model_1_test = model_1_data.iloc[model_1_split_index:] # 20% of rows for test set

#Model 2 (APPL + Others)
model_2_split_index = int(len(model_2_data) * 0.8) # 80% index for train set
model_2_train = model_2_data.iloc[:model_2_split_index] # 80% of rows for train set
model_2_test = model_2_data.iloc[model_2_split_index:] # 20% of rows for test set

# separating features (x) and target (y)
# The model will learn from features (x) to predict the binary target (y)

# Model 1
x_train_1 = model_1_train.drop(columns='target') # Features input for training (daily_change, volume_change, rolling_close_mean, rolling_volume_mean are independent variables)
y_train_1 = model_1_train['target'] # output for training (dependent variable (0 or 1))
x_test_1 = model_1_test.drop(columns='target')
y_test_1 = model_1_test['target']

# Model 2
x_train_2 = model_2_train.drop(columns='target') # Features input for training (daily_change, volume_change, rolling_close_mean, rolling_volume_mean are independent variables)
y_train_2 = model_2_train['target']
x_test_2 = model_2_test.drop(columns='target')
y_test_2 = model_2_test['target']


# The shapes of eac dataset
print('Model 1 ( AAPL only ):')
print(f'Training set: {x_train_1.shape}, Testing set: {x_test_1.shape}')
print('Model 2 ( AAPL + Others ):')
print(f'Training set: {x_train_2.shape}, Testing set: {x_test_2.shape}')

## Models Implementation

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Model 1: AAPL only
scaler1 = StandardScaler()
x_train_1_scaled = scaler1.fit_transform(x_train_1)
x_test_1_scaled = scaler1.transform(x_test_1)

clf1 = LogisticRegression(max_iter=1000)
clf1.fit(x_train_1_scaled, y_train_1)
y_prediction_1 = clf1.predict(x_test_1_scaled)

print(accuracy_score(y_test_1, y_prediction_1))
print(classification_report(y_test_1, y_prediction_1))


# Model 2: 10 IT sector companies
scaler2 = StandardScaler()
x_train_2_scaled = scaler2.fit_transform(x_train_2)
x_test_2_scaled = scaler2.transform(x_test_2)

clf2 = LogisticRegression(max_iter=1000)
clf2.fit(x_train_2_scaled, y_train_2)
y_prediction_2 = clf2.predict(x_test_2_scaled)

print(accuracy_score(y_test_2, y_prediction_2))
print(classification_report(y_test_2, y_prediction_2))